In [9]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Função para criar a pasta de saída
def create_output_directory():
    output_dir = 'figuras'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    return output_dir

# Função para construir as matrizes de massa e rigidez
def construct_matrices(N, m=1.0, k=1.0, defect=False):
    M = m * np.eye(N)  # Matriz de massa (N x N)
    K = np.zeros((N, N))  # Matriz de rigidez (N x N)
    
    # Definição de k para as interações entre massas vizinhas
    for i in range(N):
        K[i, i] = 2 * k
        K[i, (i + 1) % N] = -k  # Interação com a próxima massa
        K[i, (i - 1) % N] = -k  # Interação com a massa anterior
    
    if defect:  # Defeito de massa no centro
        m2 = 5 * m
        M[N // 2, N // 2] = m2
    
    return M, K

# Função para calcular frequências e modos
def calculate_frequencies_and_modes(M, K):
    eigenvalues, eigenvectors = np.linalg.eigh(np.linalg.inv(M) @ K)  # Diagonaliza o sistema
    frequencies = np.sqrt(np.abs(eigenvalues))  # Frequências naturais (ω)
    omega_squared = eigenvalues  # ω²
    return frequencies, eigenvectors, omega_squared

# Função para plotar e salvar os modos normais
def plot_modes(eigenvectors, frequencies, num_modes=5, mode_type='low', N=100, output_dir='figuras'):
    if mode_type == 'low':
        modes_to_plot = np.argsort(frequencies)[:num_modes]  # Menores frequências
    else:
        modes_to_plot = np.argsort(frequencies)[-num_modes:]  # Maiores frequências
    
    # Ajustando o layout para mostrar os modos de forma mais legível
    cols = 3  # Definindo 3 colunas para os subgráficos
    rows = num_modes // cols + (num_modes % cols > 0)  # Ajusta o número de linhas

    plt.figure(figsize=(12, 4 * rows))  # Ajuste do tamanho da figura
    for i, mode_index in enumerate(modes_to_plot):
        plt.subplot(rows, cols, i + 1)
        
        displacement = eigenvectors[:, mode_index]
        displacement = displacement / np.max(np.abs(displacement))  # Normaliza para o valor máximo

        plt.plot(displacement, label=f'Modo {mode_index + 1}')
        plt.title(f'Modo {mode_index + 1}\n(Frequência = {frequencies[mode_index]:.2f} Hz)')
        plt.xlabel('Índice da Massa')
        plt.ylabel('Deslocamento Relativo')
        plt.grid(True)
        plt.tight_layout()

    # Ajuste para não sobrepor os gráficos
    plt.subplots_adjust(hspace=0.5, wspace=0.3)
    # Salvar a imagem na pasta 'figuras'
    filename = os.path.join(output_dir, f"modos_{mode_type}_N{N}.png")
    plt.savefig(filename)
    plt.close()

# Função para plotar e salvar o histograma de omega^2
def plot_histogram(omega_squared, N, defect=False, output_dir='figuras'):
    plt.figure(figsize=(8, 6))
    plt.hist(omega_squared, bins=7, density=True, color='skyblue', edgecolor='black', alpha=0.7)
    
    title = 'Densidade de Estados (Histograma de ω²)'
    if defect:
        title += ' - Com Defeito de Massa'
    else:
        title += ' - Sem Defeito de Massa'
    
    plt.title(title)
    plt.xlabel('ω² (Hz²)')
    plt.ylabel('Densidade de Estados')
    plt.grid(True)
    # Salvar a imagem na pasta 'figuras'
    filename = os.path.join(output_dir, f"histograma_N{N}_{'defeito' if defect else 'homogeneo'}_omega2.png")
    plt.savefig(filename)
    plt.close()

# Função principal para calcular e plotar os resultados
def analyze_system(N, defect=False):
    output_dir = create_output_directory()  # Criar pasta de saída
    
    # Construir as matrizes de massa e rigidez
    M, K = construct_matrices(N, defect=defect)
    
    # Calcular as frequências naturais e os modos normais
    frequencies, eigenvectors, omega_squared = calculate_frequencies_and_modes(M, K)
    
    # Plotar e salvar os modos normais
    plot_modes(eigenvectors, frequencies, num_modes=5, mode_type='low', N=N, output_dir=output_dir)
    plot_modes(eigenvectors, frequencies, num_modes=5, mode_type='high', N=N, output_dir=output_dir)
    
    # Plotar e salvar o histograma de omega^2
    plot_histogram(omega_squared, N, defect=defect, output_dir=output_dir)

# Testando para um exemplo específico
analyze_system(10000, defect=False)  # Sem defeito
analyze_system(10000, defect=True)   # Com defeito
